In [1]:
import os
from notion_client import Client

def get_image_urls(page_id):
    try:
        # Initialize Notion client
        notion = Client(auth=os.environ.get("NOTION_TOKEN"))

        # Retrieve the page
        page = notion.pages.retrieve(page_id)

        # Get list of image URLs
        image_urls = []
        children = notion.blocks.children.list(block_id=page_id).get("results")
        for child in children:
            if child["type"] == "image":
                if child["image"].get("type") == "external":
                    image_urls.append(child["image"]["external"]["url"])
                elif child["image"].get("type") == "file":
                    image_urls.append(child["image"]["file"]["url"])
            elif child['type'] == "embed":
                image_urls.append(child["embed"]['url'])
        return image_urls

    except Exception as e:
        # Handle API limit or other exceptions here
        print(f"An error occurred: {str(e)}")
        return []

def update_ai_name(self, page_id, new_name):
    logging.info(f"Updating 'AiName' for page {page_id}...")
    update_properties = {
        "AiName": {
            "title": [
                {
                    "text": {
                        "content": new_name
                    }
                }
            ]
        }
    }
    response = self.notion.pages.update(page_id, properties=update_properties)
    if response:
        logging.info(f"Successfully updated 'AiName' for page {page_id}!")
    else:
        logging.info(f"Failed to update 'AiName' for page {page_id}.")

import os
import requests
import logging
from PIL import Image
from io import BytesIO
from bardapi import Bard

# Set up logging configuration
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


def process_images(images_urls):
    # Configure API key
    api_key = "ZAiB9lhD1HnhRZFS3ieaM_QCtIwC-BRO4B4HcRGakEKl1tEo3sW9Ir4o8ZL0YK81OQWsRQ."  # Replace with your actual API key
    os.environ["_BARD_API_KEY"] = api_key

    # Initialize the Bard API and session
    session = requests.Session()
    session.headers = {
        "Host": "bard.google.com",
        "X-Same-Domain": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 11.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
        "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8",
        "Origin": "https://bard.google.com",
        "Referer": "https://bard.google.com/",
    }
    session.cookies.set("__Secure-1PSID", os.getenv("_BARD_API_KEY"))

    bard = Bard(token=api_key, session=session, timeout=30)
    for index, url in enumerate(images_urls):
        response = requests.get(url)
        with open(f"images/temp_{index}.jpg", "wb") as f:
            f.write(response.content)
        image_path = f"images/temp_{index}.jpg"
        logger.info(f"Downloaded image: {image_path}")

        with open(image_path, "rb") as image_file:
            img = image_file.read()
        response = bard.ask_about_image(
            f"Screenshot {index} Rule 1: max sentences = 3 Rule 2: Output must contain only the name of the topic Rule 3: Output must be in Ukrainian. Rule 4: Topic must be bold.",
            img,
        )["content"]
        logger.info(f"Bard API response for image {index}: {response}")

    # Now that we have processed all images, we can use the Bard API to get the short description
    response = bard.get_answer(
        "So can you give a short description about this lesson? Rule 1: max sentences = 1 Rule 2: Output must contain only the name of the topic Rule 3: Output must be in Ukrainian. Rule 4: Topic must be bold."
    )["content"]
    response = response[response.index("**") + 2 : response.rindex("**")]
    logger.info(f"Short description generated: {response}")
    return response
